** 基于inception-resnet-v2对dog vs cat图片训练好的特征，去除全连接网络，重新训练 **
* 模型训练特征 gap_inception_resnet_v2.h5

In [ ]:
from bottleneck_ir2 import create_inception_resnet_v2
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py


import warnings
warnings.filterwarnings('ignore')

model=create_inception_resnet_v2()
model.load_weights('weight-v1.h5',by_name=True)
width = 299
height = 299
input_tensor = Input((height, width, 3))
x = input_tensor
model = Model(model.input, GlobalAveragePooling2D()(model.output))

gen = ImageDataGenerator()
train_generator = gen.flow_from_directory("train2", (width,height), shuffle=False,
                                              batch_size=16)
test_generator = gen.flow_from_directory("test2", (width,height), shuffle=False,
                                             batch_size=16, class_mode=None)
train = model.predict_generator(train_generator, train_generator.nb_sample)
test = model.predict_generator(test_generator, test_generator.nb_sample)
with h5py.File("gap_inception_resnet_v2.h5") as h:
     h.create_dataset("train", data=train)
     h.create_dataset("test", data=test)
     h.create_dataset("label", data=train_generator.classes)